In [ ]:
'''
This smippit can be used to train the model
'''
# Importing packages


from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from torch.utils.data import DataLoader, Sampler


In [ ]:
# Creating Dataloaders
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")




data_transforms = {
    'train': transforms.Compose([
        # transforms.Resize(224),
        # transforms.RandomHorizontalFlip(),
        #transforms.RandomResizedCrop((224,224)),
        #transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Resize((224,224)),
        transforms.Normalize([0.5], [0.5])
    ]),
    'val': transforms.Compose([
        # transforms.Resize(224),
        # transforms.CenterCrop(224),
        #transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Resize((224,224)),
        transforms.Normalize([0.5], [0.5])
    ]),
}


# npy-loader in case you want to read images in .npy
def npy_loader(path):
    sample = torch.from_numpy(np.load(path))
    return sample

data_dir = 'PATH'
#image_datasets = {x: datasets.DatasetFolder(os.path.join(data_dir, x), npy_loader, (('.spec')),
#                                          data_transforms[x])
#                  for x in ['train', 'val']}
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=8, shuffle=True
                                             )
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes



In [ ]:
# Plotting te sample plots just for checking the image quality
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    #mean = np.array([0.485, 0.456, 0.406])
    #std = np.array([0.229, 0.224, 0.225])
    #inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])






In [ ]:
# Defining functions for save and load checkpoints in case of the abrupt kill of code

def save_checkpoint(state, filename='PATH/my_checkpoint_resnet18_img_finetune_3_classes.pth.tar'):
  torch.save(state, filename)


def load_checkpoint(checkpoint):
  model_ft.load_state_dict(checkpoint['state_dict'])
  optimizer_ft.load_state_dict(checkpoint['optimizer'])


        
# Defining a function for the training loop

def train_model(model, criterion, optimizer, scheduler, num_epochs=20):
    since = time.time()
    load_model= False

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0


    if load_model:
      load_checkpoint(torch.load('PATH/my_checkpoint_resnet18_img_finetune_3_classes.pth.tar'))



    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

      

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += (preds == labels).sum()
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            

            if epoch % 3==0:
              checkpoint = {
                  'epoch' : epoch,
                'state_dict' : model.state_dict(),
                'optimizer' : optimizer.state_dict()
                }
              save_checkpoint(checkpoint)

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model









In [ ]:
# Process of Transfer Learning using resnet18       
        
model_ft = models.resnet18(pretrained=True)

num_ftrs = model_ft.fc.in_features
#model_ft.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).

for param in model_ft.parameters():
  param.requires_grad=True

model_ft.fc = nn.Sequential(nn.Dropout(0.5),nn.Linear(num_ftrs, 3))

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum= 0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.1)





model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=20)





In [ ]:
# Creation of confusion matrix to better understand the classification. Since 'nb_classes' represents the value of 3 as we are implementing the call
# classification for 'ph', 'sm' and 'phtb'. In case if you ant to extend the call types change the 'nb_calsses' value accordingly.

nb_classes = 3

confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for i, (inputs, classes) in enumerate(dataloaders['train']):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model_ft(inputs)
        preds = torch.argmax(outputs, dim=1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

print(confusion_matrix)
print(confusion_matrix.diag()/confusion_matrix.sum(1))


nb_classes = 3

confusion_matrix_val = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for i, (inputs, classes) in enumerate(dataloaders['val']):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model_ft(inputs)
        preds = torch.argmax(outputs, dim=1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix_val[t.long(), p.long()] += 1


print(confusion_matrix_val)
print(confusion_matrix_val.diag()/confusion_matrix_val.sum(1))